In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

Combining the different dataframes

In [ ]:
import pandas as pd

# set files path
ghg1 = '../input/2015-to-2019-ghg-data/Greenhouse Gas Emissions_2015.csv'
ghg2 = '../input/2015-to-2019-ghg-data/Greenhouse Gas Emissions_2016.csv'
ghg3 = '../input/2015-to-2019-ghg-data/Greenhouse Gas Emissions_2017.csv'
ghg4 =  '../input/2015-to-2019-ghg-data/Greenhouse Gas Emissions_2018.csv'
ghg5= '../input/2015-to-2019-ghg-data/Greenhouse Gas Emissions_2019.csv'

print("*** Merging multiple csv files into a single pandas dataframe ***")

# merge files
dataFrame = pd.concat(
   map(pd.read_csv, [ghg1, ghg2, ghg3, ghg4, ghg5]), ignore_index=True)
print(dataFrame)

In [ ]:
print("The shape of the dataset is : ", dataFrame.shape)

In [ ]:
dataFrame.head()

In [ ]:
dict = {}
for i in list(dataFrame.columns):
    dict[i] = dataFrame[i].value_counts().shape[0]

pd.DataFrame(dict,index=["unique count"]).transpose()

Classifying into different columns for analysis afterwards

In [ ]:
cat_cols = ['Company name and facility','Town','Province']
con_cols = ["Year","NPRI","NAICS","CO2","CH4","N20","HFC","PFC", "SF6"]
target_col = ["Total"]
print("The categorial cols are : ", cat_cols)
print("The continuous cols are : ", con_cols)
print("The target variable is :  ", target_col)

my_cols= cat_cols + target_col

In [ ]:
dataFrame[con_cols].describe().transpose()

Missing values


In [ ]:
dataFrame.isnull().sum()

Replaces NAN values with 0

In [ ]:
dataFrame.fillna(0)

In [ ]:
dataFrame.dtypes

Importing the various libraries for testing afterwards

In [ ]:
# check for dupicates
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split


In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score

from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score

Defining a function to convert the various c02 values

useful resources
https://towardsdatascience.com/forecasting-atmospheric-co2-concentration-with-python-c4a99e4cf142

https://www.canadalearningcode.ca/lessons/co2-trends-with-python/

https://www.epa.gov/energy/greenhouse-gas-equivalencies-calculator

Hans code on converting a function

In [ ]:
dataFrame.head()

In [ ]:
data_total = dataFrame[["CH4","N20","HFC","PFC","SF6"]]


In [ ]:
def Convert_CO2(data,GWP):
  result = np.zeros(np.shape(data))
  for i in range(5):
    result[:,i] = data.iloc[:,i]*float(GWP[i])*0.001
    data.iloc[:,i] = result[:,i]
  return data

Getting the right values for C02 emissions and making a new column

In [ ]:
GWP_CH4 = 25
GWP_N2O = 298
GWP_SF6 = 23900
GWP_PFC = 7390
GWP_HFC = 3500
GWP = [GWP_CH4,GWP_N2O,GWP_HFC,GWP_PFC,GWP_SF6]

Getting the values from Hans code notebook

In [ ]:
finaldata_CO2 = Convert_CO2(data_total.iloc[:,8:12], GWP)

break testing the other preprocessing methods

In [ ]:
dataFrame= df

df = df_clean


In [ ]:
df = dataFrame.drop(['PDGES ID'], axis=1)

In [ ]:
df
df.fillna(0)

In [ ]:
df_clean = df

In [ ]:
from sklearn.preprocessing import OneHotEncoder
one_hot_encoder = OneHotEncoder(sparse=False)

In [ ]:
one_hot_encoder.fit(df)


In [ ]:
one_hot_encoder.transform(df)

In [ ]:
scaler = MinMaxScaler(feature_range=(0,1)) 

#assign scaler to column:
df_scaled = pd.DataFrame(scaler.fit_transform(df_clean), columns=df_clean.columns)

In [ ]:
df = dataFrame.drop(['Company name and facility','Town', 'Province','PDGES ID'], axis=1)
df

In [ ]:
df['NPRI'] = df['NPRI'].fillna(0)
df

In [ ]:
df

In [ ]:
scaler = MinMaxScaler(feature_range=(0,1)) 

#assign scaler to column:
df_scaled = pd.DataFrame(scaler.fit_transform(df), columns=df.columns)

In [ ]:
df_scaled.describe()
df.describe()

In [ ]:
# clarify what is y and what is x label
y = df_scaled['Total']
X = df_scaled.drop(['Total'], axis = 1)

# divide train test: 80 % - 20 %
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=29)

In [ ]:
len(X_train)
len(X_test)

model pipeline

In [ ]:
# Shuffle df
#shuffled_df = df_scaled.sample(frac=1,random_state=4)

                               # Put all the fraud class in a separate dataset.
#tot_df = shuffled_df.loc[shuffled_df['Total'] == 1]

#Randomly select 492 observations from the non-fraud (majority class)
#non_tot_df = shuffled_df.loc[shuffled_df['Total'] == 0].sample(n=400,random_state=42)

# Concatenate both dataframes again
#normalized_df = pd.concat([tot_df, non_tot_df])

In [ ]:
y_train = df['Total']
X_train = df.drop('Total', axis=1)

from sklearn.pipeline import Pipeline

classifiers = [LogisticRegression(),SVC(),DecisionTreeClassifier(),KNeighborsClassifier(2)]

for classifier in classifiers:
    pipe = Pipeline(steps=[('classifier', classifier)])
    pipe.fit(X_train, y_train)   
    print("The accuracy score of {0} is: {1:.2f}%".format(classifier,(pipe.score(X_test, y_test)*100)))

In [ ]:
# search for optimun parameters using gridsearch
params= {'n_neighbors': np.arange(1, 10)}
grid_search = GridSearchCV(estimator = KNeighborsClassifier(), param_grid = params, 
                           scoring = 'accuracy', cv = 10, n_jobs = -1)
knn_clf = GridSearchCV(KNeighborsClassifier(),params,cv=3, n_jobs=-1)

In [ ]:

# train the model
knn_clf.fit(X_train,y_train)
knn_clf.best_params_ 

In [ ]:
from sklearn.tree import DecisionTreeClassifier
dtree= DecisionTreeClassifier(random_state=7)

In [ ]:
# grid search for optimum parameters
params = {'max_features': ['auto', 'sqrt', 'log2'],
          'min_samples_split': [2,3,4,5,6,7,8,9,10,11,12,13,14,15], 
          'min_samples_leaf':[1,2,3,4,5,6,7,8,9,10,11]}
tree_clf = GridSearchCV(dtree, param_grid=params, n_jobs=-1)

In [ ]:
# train the model
tree_clf.fit(X_train,y_train)
tree_clf.best_params_ 

In [ ]:
# predictions
tree_predict = tree_clf.predict(X_test)